In [82]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from sqlalchemy import create_engine
from sqlalchemy import text
from sqlalchemy.types import Date, Float
import os


In [75]:
#Importing Data from redfin.com gathered from Multiple Listing Service (MLS) - a database of realestate listings. This dataset is the buyer versus seller dynamics from 2013 to present. We are interested in extracting the date and the seller buyers percentage difference data. The desired output shape is a two column dataframe with "Date" on the left and "Sellers Buyers Percentage Difference" on the Right. 

#We are using sheet 1 which has the data in a stagged format

home_data = pd.read_excel('buyers vs sellers.xlsx', sheet_name='Sheet 1', header=None)

In [76]:
home_data.head(30)

,0,1,2,3,4,5,6,7,8,9,...,145,146,147,148,149,150,151,152,153,154
0,NaN,NaN,Day of Period Begin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Period Begin,NaN,"January 1, 2013","February 1, 2013","March 1, 2013","April 1, 2013","May 1, 2013","June 1, 2013","July 1, 2013","August 1, 2013",...,"December 1, 2024","January 1, 2025","February 1, 2025","March 1, 2025","April 1, 2025","May 1, 2025","June 1, 2025","July 1, 2025","August 1, 2025","September 1, 2025"
2,1/1/2013,Buyers,2419833.516221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Sellers,2484572.951051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Buyers YoY,0.132721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,Sellers YoY,-0.133711,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,Seller Buyer Percentage Difference,0.026754,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2/1/2013,Buyers,NaN,2462572.043779,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,Sellers,NaN,2483740.89224,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,Buyers YoY,NaN,0.136175,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
start_indices = home_data.index[(home_data[0].notna()) & (home_data[1] == 'Buyers')].tolist()

# Extract the dates from column 0 at the start indices
dates = home_data.iloc[start_indices, 0]
dates_clean = pd.to_datetime(dates)

# The Seller Buyer Percentage Difference is always 4 rows after the Buyers row in each block
diff_indices = [idx + 4 for idx in start_indices]

# Extract the percentage differences
differences = []
for idx in diff_indices:
    row = home_data.iloc[idx]
    # The value is the only non-NaN in columns 2 onward
    val = row[2:].dropna().iloc[0]
    differences.append(val)

# Create the resulting DataFrame
result_df = pd.DataFrame({
    'date': dates_clean,
    'sb_percentage_difference': differences
})

# Optionally, print or save the result
print(result_df)

          date  sb_percentage_difference
2   2013-01-01                  0.026754
7   2013-02-01                  0.008596
12  2013-03-01                  0.038864
17  2013-04-01                  0.050231
22  2013-05-01                  0.049742
..         ...                       ...
742 2025-05-01                  0.337462
747 2025-06-01                  0.369270
752 2025-07-01                  0.358633
757 2025-08-01                  0.361118
762 2025-09-01                  0.366949

[153 rows x 2 columns]


In [91]:
result_df.to_csv("Data Cleaned.csv") # Checking output to ensure formatting is correct


In [92]:
#Create new table in database to hold the dataframe


host = "anly-615-project-anlyproject.g.aivencloud.com"
port = 23263
user = "avnadmin"
password = "AVNS_uZtAlXsQZVgdnkwXesP"
database = "defaultdb"

connection_string = f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string, connect_args={"ssl": {"ssl_mode": "REQUIRED"}}, echo=False)

with engine.connect() as conn:
    # Drop the table if it exists AKA overwrite an older one
    conn.execute(text("DROP TABLE IF EXISTS sellers_vs_buyers"))
    
    # Create the table with appropriate schema
    # 'date' as DATE (primary key)
    conn.execute(text("""
        CREATE TABLE sellers_vs_buyers (
            date DATE PRIMARY KEY,
            sb_percentage_difference DECIMAL(10,6)
        ) ENGINE=InnoDB
    """))
    
    conn.commit()


In [93]:
# Insert the DataFrame into the table
result_df.to_sql(
    'sellers_vs_buyers', 
    con=engine, 
    if_exists='append', 
    index=False, 
    dtype={
        'date': Date,
        'sb_percentage_difference': Float
    }
)

print("Table created and data inserted successfully.")

Table created and data inserted successfully.
